# 1. Importing Libraries

In [1]:
# The code was removed by Watson Studio for sharing.

In [2]:
#!pip install --upgrade "watson-developer-cloud>=2.5.1"
from watson_developer_cloud import PersonalityInsightsV3

import json
from os.path import join

import pandas as pd
from pandas.io.json import json_normalize

import numpy as np

# 2. PDF to TXT

In [3]:
#!pip install pdfminer.six
#import pdfminer
#pdf2txt.py ycombinator-sam-altman.pdf > ycombinator-sam-altman.txt

# 3. Data Files

In [4]:
#!git clone https://github.com/ssuleyma/InsideOut.git
path ='./InsideOut/Transcripts/'
files = os.listdir(path=path)
print(files)

['instagram-kevin-systrom.txt', 'facebook-mark-zuckerberg.txt', 'spotify-daniel-ek.txt', 'twitter-ev-williams.txt']


# 4. Watson Personality Insights (WPI) Authentication 

In [5]:
pi_version = "2017-10-13"
pi_api_key = ""
pi_url = "https://gateway.watsonplatform.net/personality-insights/api"

personality_insights = PersonalityInsightsV3(
    version = pi_version,
    iam_apikey = pi_api_key,
    url = pi_url
)

personality_insights.set_detailed_response(True)
personality_insights.set_default_headers({'x-watson-learning-opt-out': "true"})

# 5. Analysis by WPI

In [6]:
candidates_df = pd.DataFrame()
for i,f in enumerate(files):
    with open(join(path, f)) as profile_txt:
        profile = personality_insights.profile(profile_txt.read(),
                                               content_type='text/plain',
                                               accept='application/json',
                                               consumption_preferences=True,
                                               raw_scores=True).get_result()
        
        candidates_df = candidates_df.append(json_normalize(profile),ignore_index=True,sort=False)
        candidates_df.loc[i,'file'] = f

In [7]:
candidates_df.drop(columns=['processed_language','warnings'],inplace=True)

In [8]:
candidates_df.head()

,consumption_preferences,needs,personality,values,word_count,file
0,[{'consumption_preference_category_id': 'consu...,"[{'category': 'needs', 'name': 'Challenge', 's...","[{'category': 'personality', 'children': [{'ca...","[{'category': 'values', 'name': 'Conservation'...",2732,instagram-kevin-systrom.txt
1,[{'consumption_preference_category_id': 'consu...,"[{'category': 'needs', 'name': 'Challenge', 's...","[{'category': 'personality', 'children': [{'ca...","[{'category': 'values', 'name': 'Conservation'...",2517,facebook-mark-zuckerberg.txt
2,[{'consumption_preference_category_id': 'consu...,"[{'category': 'needs', 'name': 'Challenge', 's...","[{'category': 'personality', 'children': [{'ca...","[{'category': 'values', 'name': 'Conservation'...",1854,spotify-daniel-ek.txt
3,[{'consumption_preference_category_id': 'consu...,"[{'category': 'needs', 'name': 'Challenge', 's...","[{'category': 'personality', 'children': [{'ca...","[{'category': 'values', 'name': 'Conservation'...",2852,twitter-ev-williams.txt


In [10]:
project.save_data('candidates_df.csv',candidates_df.to_csv(index=False),overwrite=True)

{'asset_id': '36131aed-e240-4df3-a8e8-5d9963aafb07',
 'bucket_name': 'insideout-donotdelete-pr-arlsgjghrogjm7',
 'file_name': 'candidates_df.csv',
 'message': 'File candidates_df.csv has been written successfully to the associated OS'}

# 6. Data Cleaning

# 7. Exploratory Data Analysis